In [1]:
!pip install diffusers accelerate safetensors transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 103.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 122.2 MB/s eta 0:00:00


In [2]:
!pip install ngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 20.8 MB/s eta 0:00:00


In [3]:
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=95c1245232ba85b2e7851afb9da94033aa267eb711f090c8872bb25b8beb2377
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


In [4]:
!pip install flask-cors

In [ ]:
from flask import Flask, request, jsonify
from PIL import Image, ImageOps
import requests
import torch
from io import BytesIO
from diffusers import StableDiffusionInstructPix2PixPipeline, EulerAncestralDiscreteScheduler
from pyngrok import ngrok
import base64
from flask_cors import CORS

app = Flask(__name__)
CORS(app)
# Load the model and set it up
model_id = "timbrooks/instruct-pix2pix"
pipe = StableDiffusionInstructPix2PixPipeline.from_pretrained(model_id, torch_dtype=torch.float16, safety_checker=None)
pipe.to("cuda")
pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)

# Download image from URL and preprocess
def download_image(url):
    image = Image.open(requests.get(url, stream=True).raw)
    image = ImageOps.exif_transpose(image)
    image = image.convert("RGB")
    return image

# Define API endpoint

@app.route('/process_image', methods=['POST'])
def process_image():
    try:
        data = request.get_json()
        url = data.get('url', None)
        prompt = data.get('prompt', None)

        if url is None or prompt is None:
            return jsonify({'error': 'Missing URL or prompt'}), 400

        image = download_image(url)

        images = pipe(prompt, image=image, num_inference_steps=10, image_guidance_scale=1).images

        processed_image = images[0]

        buffered = BytesIO()
        processed_image.save(buffered, format="PNG")  # You can replace "PNG" with "JPEG" if you prefer JPEG format
        image_bytes = buffered.getvalue()

        # Encode the image bytes in base64
        image_base64 = base64.b64encode(image_bytes).decode("utf-8")

        # Create JSON response
        response_data = {
            'processed_image_base64': image_base64
        }

        return jsonify(response_data), 200

    except Exception as e:
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    # Set ngrok auth token
    ngrok.set_auth_token("2UCBnO3oWGPJqVUOcUQ1yDWtUCY_4eizetwxTW6ZoesLiA2wG")
    # Start ngrok tunnel
    public_url = ngrok.connect(5000)
    print('Public URL:', public_url)
    app.run(host='0.0.0.0', port=5000)

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

Public URL: NgrokTunnel: "https://ed71-34-66-167-173.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [19/Aug/2023 08:31:24] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [19/Aug/2023 08:41:54] "OPTIONS /process_image HTTP/1.1" 200 -


  0%|          | 0/10 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [19/Aug/2023 08:42:06] "POST /process_image HTTP/1.1" 200 -
